In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from gioutils.utils import parse_offset, today, get_next_n, get_bday
from gioutils.gui.base import View, ViewStore, WidgetView, Tabs
from gioutils.blpw import BlpQuery
from gioutils.utils import RepeatedTimer, RepeatedTimerThread
from gioutils import ezutils as ez
from gioutils.ezutils import bh, bhs, SQL, ZCILSHelper, get_price_info, get_id, LiveInflLocal, COBInflLocal
from gioutils.gui.base import ViewStore
from gioutils.blpw import BlpParser
from gioutils.gui.datagrid import DataGridGio

from api.data.base import data_path, radar_path
from api.gui.params import bbg_params, spot_params, fwd_params
from api.gui.theme import bg_color, grid_style
from api.collectors import CollectorStoreMixin, SpotForward, Futures, Market, Mtgs, FwdMatrix, VolMatrix, ECOPred, ZCILMatrix, ECO
from api.gui.editable_grid import EditableGrid
from api.gui.price_grids import PricingGrid
from api.gui.grids import SpotGrid, FutureGrid, MtgGrid, MatGrid, GroupMktGrid # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.global_config import GlobalConfig
from api.gui.monitors import MktMonitor, VolMonitor, ECOMonitor, MktMonitorMenashe
from api.app import DataCollector
from api.gui.renderers import RendererStoreMixin, RendererComponentStoreMixin


pd.options.plotting.backend = "plotly"

In [4]:
import clarion
from gioutils.clarion_ext import super_clarion

clarion.server_login()

2022-12-13 16:55:55,115 - Authenticating...
2022-12-13 16:55:55,907 - Successfully authenticated


In [5]:
super_clarion(clarion)

In [6]:
clarion.price_metrics().sort_values('metricType')[100:]

metricType assetClass  \
53       ReferenceFuture     Equity   
54       ReferenceFuture     Equity   
55       ReferenceFuture     Equity   
56                   Rho     Equity   
58                   Rho     Equity   
57                   Rho     Equity   
30         RunningSpread     Credit   
94                Saving         Fx   
7         SettlementDate        Any   
59                  Skew     Equity   
60                 Smile     Equity   
61                  Spot     Equity   
95                  Spot         Fx   
137                 Spot      Rates   
138            StartDate      Rates   
139            StartDate      Rates   
62             StartDate     Equity   
8                 Strike        Any   
140            SwapAnn01      Rates   
63                 Theta     Equity   
9              TimeValue        Any   
10             TimeValue        Any   
64                 Vanna     Equity   
96                 Vanna         Fx   
65         VarswapSpread     Equity   
66                  Vega     Equity   
97                  Vega         Fx   
98                  Vega         Fx   
31                  Vega     Credit   
141                 Vega      Rates   
143           Volatility      Rates   
147           Volatility      Rates   
144           Volatility      Rates   
146           Volatility      Rates   
142           Volatility      Rates   
145           Volatility      Rates   
99            Volatility         Fx   
104           Volatility         Fx   
103           Volatility         Fx   
102           Volatility         Fx   
101           Volatility         Fx   
100           Volatility         Fx   
70            Volatility     Equity   
69            Volatility     Equity   
68            Volatility     Equity   
67            Volatility     Equity   
71                 Volga     Equity   
105                Volga         Fx   
148                Yield      Rates   
149  YieldYieldAssetSwap      Rates   
150              Zspread      Rates   
151              Zspread      Rates   

                                                config  
53                              {'Type': 'Instrument'}  
54                                  {'Type': 'Future'}  
55                                   {'Type': 'Level'}  
56   {'Type': 'Total', 'VolSpotDynamic': 'StickyStr...  
58   {'Type': 'Borrow', 'VolSpotDynamic': 'StickySt...  
57   {'Type': 'Discount', 'VolSpotDynamic': 'Sticky...  
30                                                      
94                                                      
7                                                       
59                                                      
60                                                      
61                                 {'Type': 'Implied'}  
95                                                      
137                                                     
138                                 {'Type': 'Bumped'}  
139                               {'Type': 'Unbumped'}  
62                                                      
8                                                       
140                                                     
63   {'TimeBumpSize': '1', 'SpotTimeDynamics': 'Div...  
9                                      {'Unit': 'USD'}  
10                             {'Unit': 'BasisPoints'}  
64                  {'VolSpotDynamic': 'StickyStrike'}  
96    {'Type': 'ATM', 'VolSpotDynamic': 'StickyDelta'}  
65                                                      
66                                                      
97                                     {'Type': 'ATM'}  
98              {'Type': 'ATM', 'Scaling': 'RootTime'}  
31                                                      
141                                {'Period': 'Daily'}  
143                {'Type': 'ATM', 'Period': 'Annual'}  
147               {'Type': 'Skew', 'Period': 'Annual'}  
144              {'Type': 'Strike', 'Period': 'Daily'}  
146     

In [7]:
# Create a list of metrics in form clarion.metric('label', 'metricType', 'assetClass', {config})
# 'label' here is an arbitrary string that can be used to identify the metric
pv = clarion.metric('pv', 'PresentValue', 'Any')
strike = clarion.metric('strike', 'Strike', 'Any')
maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'})
start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'})

fx_price = clarion.metric('Price', 'Price', 'Fx', {'Unit': 'PercentLocal'}) 
atm_vol = clarion.metric('AtmVol', 'Volatility', 'Fx', {'Type': 'ATM'})
strike_vol = clarion.metric('StrikeVol', 'Volatility', 'Fx', {'Type': 'Strike'})
carry_t1 = clarion.metric('T+1Carry', 'Carry', 'Fx', {'Period': '1', 'StartBumpDays': '1'})


fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates')
strike_vol = clarion.metric('strike_vol', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'})

# m8 = clarion.metric('EqDelta', 'Delta', 'Equity', {'VolSpotDynamic': 'StickyStrike'})
# m9 = clarion.metric('EqPrice', 'Price', 'Equity', {'Unit': 'Local'})

fx_metrics = [pv, fx_price, atm_vol, strike_vol, carry_t1]
rates_metrics = [start, maturity, pv, strike, fwd_rate, strike_vol]
# eq_metrics = [m1,m8,m9]

## Find which products are supported and required parameters

In [9]:
instrument_types = clarion.price.instrument_types()
instrument_types;

['CreditBond',
 'CreditDefaultSwap',
 'CreditDefaultSwapIndex',
 'CreditDefaultSwapIndexOption',
 'EquityCFD',
 'EquityCashOption',
 'EquityConditionalVarSwap',
 'EquityDividendFuture',
 'EquityFuture',
 'EquityFutureOption',
 'EquityStock',
 'EquityVarSwap',
 'EquityVolSwap',
 'FxDigi',
 'FxDoubleKnockIn',
 'FxDoubleKnockInDigi',
 'FxDoubleKnockInDigiWindow',
 'FxDoubleKnockInWindow',
 'FxDoubleKnockOut',
 'FxDoubleKnockOutDigi',
 'FxDoubleKnockOutDigiWindow',
 'FxDoubleKnockOutWindow',
 'FxDoubleNoTouch',
 'FxDoubleOneTouch',
 'FxEuropeanKnockIn',
 'FxEuropeanKnockOut',
 'FxEuropeanReverseKnockIn',
 'FxEuropeanReverseKnockOut',
 'FxForward',
 'FxKnockIn',
 'FxKnockInDigi',
 'FxKnockInDigiWindow',
 'FxKnockInWindow',
 'FxKnockOut',
 'FxKnockOutDigi',
 'FxKnockOutDigiWindow',
 'FxKnockOutWindow',
 'FxNoTouchDown',
 'FxNoTouchUp',
 'FxOneTouchDown',
 'FxOneTouchKnockOutUpInDownOut',
 'FxOneTouchKnockOutUpOutDownIn',
 'FxOneTouchUp',
 'FxReverseKnockIn',
 'FxReverseKnockInDigi',
 'FxReve

In [10]:
clarion.price.instrument_type_params('FxVanilla')

[{'field': 'CurrencyPair', 'optional': False},
 {'field': 'Expiry', 'optional': False},
 {'field': 'Strike', 'optional': False},
 {'field': 'OptionType', 'optional': False},
 {'field': 'Notional', 'optional': True}]

### Full Python Code - Rates Swap pricing

In [11]:
tenors = [
    'o/n',
    '1m', 
    '3m', 
    '6m', 
    '1y', 
    '2y', 
    '3y', 
    '4y', 
    '5y', 
    '6y', 
    '7y', 
    '8y', 
    '9y', 
    '10y',
    '12y',
    '15y',
    '20y',
    '25y',
    '30y',
    '40y',
    '50y',
]

In [12]:
fwd_starts = [
    None,
    '1m',
    '3m',
    '6m',
    '1y',
    '2y',
    '3y',
    '4y',
    '5y',
    '6y', 
    '7y', 
    '8y', 
    '9y', 
    '10y',
]

In [13]:
offsets = [
    '1m',
    '3m',
    '6m',
    '1y'
]

In [14]:
swap_floating = 'SOFR A'
ccy = 'USD'

In [15]:
trade1 = clarion.instruments.FxVanilla(
    currency_pair='EURUSD',
    expiry='3M',
    strike='a',
    option_type='C'
)

In [19]:
clarion.price.run(
    fx_metrics,
    trade1.trade,
    start_date='2018-01-01',
    end_date='2022-12-13',
    market_mode='live'
)

2022-12-13 17:02:00,604 - Requesting data
2022-12-13 17:02:05,573 - Request complete
2022-12-13 17:02:05,717 - Response Count Summary - DFRAME:1292, IS_ERROR:1, HAS_ERROR:1


{'DFRAME':                                               id periodType    description  \
 0     fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
 1     fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
 2     fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
 3     fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
 4     fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
 ...                                          ...        ...            ...   
 1287  fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
 1288  fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
 1289  fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
 1290  fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
 1291  fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
 
         period    AtmVol     Price  T+1

In [21]:
df

id periodType    description  \
0     fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
1     fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
2     fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
3     fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
4     fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
...                                          ...        ...            ...   
1287  fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
1288  fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
1289  fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
1290  fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   
1291  fxvanilla_478ae5ead34d4772bda69ff83906af3b      daily  EURUSD C A 3M   

        period    AtmVol     Price  T+1Carry        pv  
0     20180101  0.070597  0.013409 -0.000108  0.016005  
1     20180102  0.073172  0.014199 -0.000216  0.017102  
2     20180103  0.071461  0.013874 -0.000170  0.016681  
3     20180104  0.069349  0.013547 -0.000149  0.016357  
4     20180105  0.067313  0.013087 -0.000121  0.015751  
...        ...       ...       ...       ...       ...  
1287  20221207  0.094030  0.018019 -0.000202  0.018927  
1288  20221208  0.092020  0.017736 -0.000137  0.018703  
1289  20221209  0.089789  0.017224 -0.000655  0.018156  
1290  20221212  0.089007  0.016894 -0.000144  0.017779  
1291  20221213  0.083513  0.016054 -0.000954  0.017067  

[1292 rows x 8 columns]

In [ ]:
clarion.price.create_trade('RatesSwap', )

In [ ]:
swap_trades = []
for start in fwd_starts:
    for tenor in tenors:
        trade = clarion.instruments.RatesSwap(
            currency=ccy,
            instrument_name=swap_floating,
            strike='a',
            tenor=tenor,
            start_date=start,
            swap_type='Pay',
            # notional='d20000',
            clearing_house='LCH',
            id=f"{start}x{tenor}" if start else f"spotx{tenor}"
        )
        swap_trades.append(trade.trade)


In [ ]:
end_date = today()
offset = '1y'

In [ ]:
data_list.__len__()

In [ ]:
df = clarion.runner.run(
    rates_metrics,
    swap_trades,
    market_mode='live',
    start_date=end_date - parse_offset(offset),
    end_date=end_date,
    # system_time=system_time,
    # polling_count=10,
    # polling_interval=1
)

In [ ]:
df

In [ ]:
clarion.runner._reqs[0]

In [ ]:
clarion.price.run(

In [ ]:
len(swap_trades)

In [ ]:
res = clarion.price.retrieve(
    req['KEY'],
    polling_count=10, 
    polling_interval=1
)

In [ ]:
req

In [ ]:
res = clarion.price.retrieve(req['KEY'], polling_count=10)

In [ ]:
res.keys()

In [ ]:
res['TIMEOUT']

In [ ]:
d = res['DFRAME']

In [ ]:
res = clarion.price.retrieve(req['KEY'])

In [ ]:
res['DFRAME']

In [ ]:
def run_multiple();

In [ ]:
swap_curves = res['DFRAME'].sort_values(['period', 'start_date', 'maturity_date'])
periods = swap_curves.id.str.split('x')
swap_curves = swap_curves.assign(
    start=periods.str[0],
    tenor=periods.str[1],
    date=pd.to_datetime(swap_curves['period'])
)

In [ ]:
date = today()

In [ ]:
swap_curves.query(f"date=='{date:%Y-%m-%d}'").plot(x='tenor', y='strike', color='start', title=swap_floating, height=500, template='plotly_white')

In [ ]:
fra_floating = 'LIBOR 3M'
ccy = 'USD'

In [ ]:
fra_trades = []
for start in fwd_starts:
    trade = clarion.instruments.RatesFra(
        currency=ccy,
        instrument_name=fra_floating,
        strike='a',
        start_date=start,
        fra_type='Pay',
        # notional='d20000',
        clearing_house='LCH',
        id=f"{start}x3m" if start else f"spotx3m"
    )
    fra_trades.append(trade.trade)

In [ ]:
offsets = [
    '1m',
    '3m',
    '6m',
    '1y'
]

In [ ]:
end_date = today()
offset = '1y'

In [ ]:
response = clarion.price.run(rates_metrics, fra_trades, market_mode='live', start_date=f"{end_date - parse_offset(offset):%Y-%m-%d}", end_date=f"{end_date:%Y-%m-%d}")

In [ ]:
fra_curves = response['DFRAME'].sort_values(['period', 'start_date', 'maturity_date'])
periods = fra_curves.id.str.split('x')
fra_curves = fra_curves.assign(
    start=periods.str[0],
    tenor=periods.str[1],
    date=pd.to_datetime(curves['period'])
)

In [ ]:
fra_curves

In [ ]:
' ' + fra_curves.query(f"date=='{date:%Y-%m-%d}'")['start']

In [ ]:
fra_curves.query(f"date=='{date:%Y-%m-%d}'").plot(x='start', y='strike', title=floating, height=500, template='plotly_white')

### Full Python Code - Rates swaption pricing

In [ ]:
rates_swaption = clarion.create_trade('swaption_1', 'ratesSwaption', {'InstrumentName': 'BBSW Q 3M',
                                                                 'Currency': 'AUD',
                                                                 'Expiry': '10-Mar-23',
                                                                 'Notional': 100000000,
                                                                 'Tenor':'6M',
                                                                 'Strike': '0.2',
                                                                 'SwaptionType': 'Pay',
                                                                 'CollateralCurrency': 'USD',
                                                                 'SettleType': 'Physical Cleared LCH',
                                                                 'PaySettle': 'Spot'})

response = clarion.price.run(rates_metrics, rates_swaption, start_date="2022-03-01",end_date="2022-03-16",market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

### Full Python Code - Equity future option pricing

In [ ]:
equity_futOpt = clarion.create_trade('id2','equityFutureOption',{'InstrumentName':'AS51 SFE',
                                                         'UnderlyingExpiry':'Sep23',
                                                         'ContractType':'OTC', 
                                                         'Expiry':'2023-09-21',
                                                         'Strike':'5500',
                                                         'OptionType':'C',
                                                         'ExerciseType':'A',
                                                         'Quantity':100})
response = clarion.price.run(eq_metrics, equity_futOpt, start_date="2022-06-01",end_date="2022-09-01",market_mode='reval')
if response['ERROR']:
    print(response['ERROR'])
    print(response['IS_ERROR'])
    print(response['HAS_ERROR'])
else:
    display(response['DFRAME'])

## Retrieve Exchange close pricing
#### Exchange close pricing is available back up to **4w history** from todays date

In [ ]:
rates_STIRFut = clarion.create_trade('stir1', 'ratesShortTermFuture', {'InstrumentName':'EURIBOR 3M ICE',
                                                                       'Currency':'EUR',
                                                                       'Tenor':'H23'})

In [ ]:
m1 = clarion.metric('FwdRate', 'Forward', 'Rates')
response = clarion.price.run([m1], rates_STIRFut, start_date='2022-09-01', end_date='2022-09-05',market_mode='reval',settings={'market_name':'exchange'})
response['DFRAME']